In [1]:
#import important packages

import pandas as pd
import numpy as np
import binance as bn
from binance.client import Client
from binance import ThreadedWebsocketManager
import json
import websocket


#### create socket url

In [2]:
symbol = 'btcusdt'
socket = f'wss://stream.binance.com:9443/ws/{symbol}@trade'

#### Trade Socket

In [3]:
# # Create an empty DataFrame globally
# df = pd.DataFrame(columns=['Event Type', 'Event Time', 'Symbol', 'Trade ID', 'Price', 'Quantity', 'Trade Time'])

# def on_message(ws, message):
#     global df  # Access the global DataFrame

#     # Convert message to dictionary
#     message_dict = json.loads(message)

#     # Safely access the expected fields
#     event_type = message_dict.get('e')  # Event type
#     event_time = message_dict.get('E')  # Event time
#     symbol = message_dict.get('s')      # Symbol (e.g., BTCUSD)
#     trade_id = message_dict.get('t')    # Trade ID
#     price = message_dict.get('p')       # Price
#     quantity = message_dict.get('q')    # Quantity
#     trade_time = message_dict.get('T')  # Trade time

#     # Check if the required fields are present
#     if event_type and event_time and symbol and trade_id and price and quantity and trade_time:
#         # Create a new row as a dictionary
#         new_data = pd.DataFrame([{
#             'Event Type': event_type,
#             'Event Time': event_time,
#             'Symbol': symbol,
#             'Trade ID': trade_id,
#             'Price': price,
#             'Quantity': quantity,
#             'Trade Time': trade_time
#         }])

#         # Append the new data to the global DataFrame
#         df = pd.concat([df, new_data], ignore_index=True)

#         # Optionally print the entire DataFrame to see all accumulated data
#         print(df)


# def on_error(ws, error):
#     print(error)

# def on_close(ws, close_status_code, close_msg):
#     print("### closed ###")

# def on_open(ws):
#     print("Opened connection")

# ws = websocket.WebSocketApp(socket,
#                             on_open=on_open,
#                             on_message=on_message,
#                             on_error=on_error,
#                             on_close=on_close)

In [4]:
# ws.run_forever()

### KAFKA PRODUCER

In [5]:
#!pip install kafka-python websocket-client
#!pip install six
#!pip install kafka-python-ng

In [ ]:
import kafka

In [7]:
from kafka import KafkaProducer

In [8]:
from kafka import KafkaProducer
import json

producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],  # Ensure this matches the advertised listener
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

producer.send('test_topic', {'key': 'value'})
producer.flush()


In [ ]:
# Kafka Producer setup
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Binance WebSocket setup
def on_message(ws, message):
    data = json.loads(message)
    print("Sending data to Kafka...")
    producer.send('binance_trades', data)  # Sending data to Kafka topic

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    print("Opened WebSocket connection")

if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://stream.binance.com:9443/ws/btcusdt@trade",
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close,
                                on_open=on_open)
    ws.run_forever()


In [ ]:
# symbol = 'BNBBTC'

# twm = ThreadedWebsocketManager()
# # start is required to initialise its internal loop
# twm.start()

# def handle_socket_message(msg):
#     print(f"message type: {msg['e']}")
#     print(msg)

#     twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)
# depth_stream_name = twm.start_depth_socket(callback=handle_socket_message, symbol=symbol)

# # some time later

# twm.stop_socket(depth_stream_name)